## Part 1: Preprocessing

In [273]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [274]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [275]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development
...,...,...
1465,No,Research & Development
1466,No,Research & Development
1467,No,Research & Development
1468,No,Sales


In [276]:
# Create a list of at least 10 column names to use as X data
x_columns = [
    'Age',
    'Department',
    'BusinessTravel',
    'JobRole',
    'DistanceFromHome',
    'Education',
    'EducationField',
    'HourlyRate',
    'WorkLifeBalance',
    'YearsAtCompany',
    'DistanceFromHome'
]


# Create X_df using your selected columns
X_df = attrition_df[x_columns]

# Show the data types for X_df
print(X_df.dtypes)



Age                  int64
Department          object
BusinessTravel      object
JobRole             object
DistanceFromHome     int64
Education            int64
EducationField      object
HourlyRate           int64
WorkLifeBalance      int64
YearsAtCompany       int64
DistanceFromHome     int64
dtype: object


In [277]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X = X_df[['Age', 'BusinessTravel', 'JobRole','DistanceFromHome', 'Education', 'Department','EducationField',
                   'HourlyRate', 'WorkLifeBalance', 'YearsAtCompany', 'DistanceFromHome']]
y = y_df['Attrition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [278]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train_encoded = pd.get_dummies(X_train, columns=['BusinessTravel','JobRole', 'Department', 'EducationField'], drop_first=True)
X_test_encoded = pd.get_dummies(X_test, columns=['BusinessTravel', 'JobRole', 'Department','EducationField'], drop_first=True)


In [279]:
print(X_train.columns)

Index(['Age', 'BusinessTravel', 'JobRole', 'DistanceFromHome',
       'DistanceFromHome', 'Education', 'Department', 'EducationField',
       'HourlyRate', 'WorkLifeBalance', 'YearsAtCompany', 'DistanceFromHome',
       'DistanceFromHome'],
      dtype='object')


In [280]:
# Create a StandardScaler
scaler = StandardScaler()


# Fit the StandardScaler to the training data
scaler.fit(X_train_encoded)


# Scale the training and testing data
X_train_scaled = scaler.transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)


In [281]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop='first')


# Fit the encoder to the training data
encoder.fit(X_train[['Department']])


# Create two new variables by applying the encoder
# to the training and testing data
department_train_encoded = encoder.transform(X_train[['Department']])
department_test_encoded = encoder.transform(X_test[['Department']])



In [282]:
# Create a OneHotEncoder for the Attrition column
encoder = OneHotEncoder(sparse_output=False, drop='first')

# Ensure y_train is reshaped into a 2D array for OneHotEncoder
attrition_train_reshaped = y_train.values.reshape(-1, 1)
attrition_test_reshaped = y_test.values.reshape(-1, 1)

# Fit the encoder to the training data using the reshaped data
encoder.fit(attrition_train_reshaped)

# Create two new variables by applying the encoder to the training and testing data
y_train_encoded = encoder.transform(attrition_train_reshaped)
y_test_encoded = encoder.transform(attrition_test_reshaped)



## Create, Compile, and Train the Model

In [283]:
# Find the number of columns in the X training data
input_dim = X_train_encoded.shape[1]
print(f"Number of input features (columns) in the training data: {input_dim}")


# Create the input layer
input_layer = layers.Input(shape=(24,), name='shared_input')


# Create at least two shared layers
input_layer = layers.Input(shape=(24,), name='shared_input')
shared_layer_1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer_2 = layers.Dense(32, activation='relu')(shared_layer_1)


Number of input features (columns) in the training data: 26


In [284]:
# Create a branch for Department
# with a hidden layer and an output layer
department_input = layers.Input(shape=(24,), name='department_input')
# Create the hidden layer
department_hidden_layer = layers.Dense(16, activation='relu')(department_input)

# Create the output layer
department_output_layer = layers.Dense(1, activation='sigmoid', name='department_output')(department_hidden_layer)



In [285]:
# Create a branch for Attrition
# with a hidden layer and an output layer
attrition_input = layers.Input(shape=(24,), name='attrition_input')

# Create the hidden layer
attrition_hidden_layer = layers.Dense(16, activation='relu')(attrition_input)


# Create the output layer
attrition_output_layer = layers.Dense(1, activation='sigmoid', name='attrition_output')(attrition_hidden_layer)



In [286]:
# Create the model
# Combine shared layers with both Department and Attrition branch outputs
combined_layer = layers.concatenate([shared_layer_2, department_output_layer, attrition_output_layer])

# Final output layer for the entire network
final_output_layer = layers.Dense(1, activation='sigmoid', name='final_output')(combined_layer)

# Define the model
model = Model(inputs=[input_layer, department_input, attrition_input],
    outputs=[final_output_layer, department_output_layer, attrition_output_layer]
)


# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'final_output': 'binary_crossentropy',
        'department_output': 'binary_crossentropy',
        'attrition_output': 'binary_crossentropy'
    },
    metrics={
        'final_output': ['accuracy'],
        'department_output': ['accuracy'],
        'attrition_output': ['accuracy']
    }
)

# Summarize the model
model.summary()

Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ shared_input (InputLayer) │ (None, 24)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_input          │ (None, 24)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_input           │ (None, 24)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_28 (Dense)          │ (None, 64)             │          1,600 │ shared_input[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_30 (Dense)          │ (None, 16)             │            400 │ department_input[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_31 (Dense)          │ (None, 16)             │            400 │ attrition_input[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_29 (Dense)          │ (None, 32)             │          2,080 │ dense_28[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 1)              │             17 │ dense_30[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 1)              │             17 │ dense_31[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_12            │ (None, 34)             │              0 │ dense_29[0][0],        │
│ (Concatenate)             │                        │                │ department_output[0][… │
│                           │                        │                │ attrition_output[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ final_output (Dense)      │ (None, 1)              │             35 │ concatenate_12[0][0]   │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,549 (17.77 KB)

 Trainable params: 4,549 (17.77 KB)

 Non-trainable params: 0 (0.00 B)

In [287]:
#Train the model
history = model.fit(
        [X_train_shared, X_train_department, X_train_attrition],
        y_train_targets,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        verbose=2
    )



Epoch 1/50
30/30 - 3s - 93ms/step - attrition_output_accuracy: 0.4138 - department_output_accuracy: 0.5372 - final_output_accuracy: 0.7670 - loss: 2.1613 - val_attrition_output_accuracy: 0.5508 - val_department_output_accuracy: 0.6525 - val_final_output_accuracy: 0.7966 - val_loss: 1.8804
Epoch 2/50
30/30 - 0s - 4ms/step - attrition_output_accuracy: 0.5351 - department_output_accuracy: 0.7223 - final_output_accuracy: 0.8404 - loss: 1.7342 - val_attrition_output_accuracy: 0.6483 - val_department_output_accuracy: 0.7331 - val_final_output_accuracy: 0.7966 - val_loss: 1.6936
Epoch 3/50
30/30 - 0s - 5ms/step - attrition_output_accuracy: 0.6553 - department_output_accuracy: 0.7936 - final_output_accuracy: 0.8404 - loss: 1.5276 - val_attrition_output_accuracy: 0.7246 - val_department_output_accuracy: 0.7627 - val_final_output_accuracy: 0.7966 - val_loss: 1.5967
Epoch 4/50
30/30 - 0s - 4ms/step - attrition_output_accuracy: 0.7436 - department_output_accuracy: 0.8213 - final_output_accuracy: 0

In [288]:
# Extract and display evaluation results
combined_loss = evaluation_results[0]
attrition_output_accuracy = evaluation_results[1]
department_output_accuracy = evaluation_results[2]
final_output_accuracy = evaluation_results[3]

# Print the evaluation results
print(f"\nEvaluation Results:")
print(f"Combined Loss: {combined_loss}")
print(f"Attrition Output Accuracy: {attrition_output_accuracy}")
print(f"Department Output Accuracy: {department_output_accuracy}")
print(f"Final Output Accuracy: {final_output_accuracy}")


Evaluation Results:
Combined Loss: 1.2119388580322266
Attrition Output Accuracy: 0.853741466999054
Department Output Accuracy: 0.8673469424247742
Final Output Accuracy: 0.8605442047119141


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy doesnt tell the full story as it doesnt capture a models full perfomance, thus id say even for this data I wouldnt solely rely on accuracy alone but incorporate other metrics such as recall, precision, and F1-score.

2. I chose sigmoid fuction for the output layer as attrition, department and final output were treated as a binary classification problem.

3. I would suggest model improvements such as optimizing parameres like number of layers and units per layer, or manupaling fetures through encoding, and scaling. It would also be interesting to combine and comapre the prediction with other models such as XGBoost.